In [1]:
import os
import pinecone
import sys
from langchain.llms import Replicate
from langchain.vectorstores import Pinecone
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders.csv_loader import CSVLoader

c:\Users\irfan\Anaconda3\envs\llama\lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
import pandas as pd
data = pd.read_csv("SPOTIFY_REVIEWS.csv")

In [23]:
data = data.dropna()
data_select = data.iloc[:100000, :]
data_select = data_select[["review_text", "review_rating", "review_likes"]]
data_select.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, 0 to 107851
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   review_text    100000 non-null  object
 1   review_rating  100000 non-null  int64 
 2   review_likes   100000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.1+ MB


In [24]:
data_select.head()

,review_text,review_rating,review_likes
0,Use it every day,5,1
1,"I enjoy the awesome UI of this app, and it has...",5,4
2,Love it! Especially the new design!,5,2
3,"Awesome UI, best music app out there!",5,1
4,As a professional Android developer I'm glad t...,5,10


In [25]:
data_select.to_csv("spotify_review_simplify.csv", index = False)

In [26]:
os.environ['REPLICATE_API_TOKEN'] = "r8_EXRCcMkZ0OIbqiyodux14JdxiiNBAIl2uN07o"

In [27]:
pinecone.init(api_key='4519c82a-62e5-4201-9bb6-a94537eeec44', environment='gcp-starter')

In [28]:
loader = CSVLoader(file_path="spotify_review_simplify.csv", encoding="utf8")
documents = loader.load()

In [29]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [30]:
embeddings = HuggingFaceEmbeddings()

c:\Users\irfan\Anaconda3\envs\llama\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [31]:
index_name = "chatbot-llm"
index = pinecone.Index(index_name)
vectordb = Pinecone.from_documents(texts, embeddings, index_name=index_name)

In [32]:
llm = Replicate(
    model="meta/llama-2-7b-chat:4f0a4744c7295c024a1de15e1a63c880d3da035fa1f49bfd344fe076074c8eea",
    input={"temperature": 0.75, "max_length": 3000}
)

Init param `input` is deprecated, please use `model_kwargs` instead.


In [33]:
qa_chain = ConversationalRetrievalChain.from_llm(
    llm,
    vectordb.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True
)

In [36]:
chat_history = []
while True:
    query = input('Prompt: ')
    if query == "exit" or query == "quit" or query == "q":
        print('Exiting')
        sys.exit()
    result = qa_chain({'question': query, 'chat_history': chat_history})
    print('Answer: ' + result['answer'] + '\n')
    chat_history.append((query, result['answer']))

Answer: Based on the two reviews provided, users appreciate the following features or aspects of your application:
1. Minimal ads: Both reviewers mention that the app has minimal ads, which is a significant aspect they appreciate.
2. Great lists: Reviewers appreciate the quality of the lists provided by the app, which they find helpful.
3. Cost: The second reviewer mentions that they would like to see a lower cost of subscription, which suggests that they appreciate the app's pricing model.

Answer: Based on the reviews, users most often compare our application with Pandora.

Answer: Based on the provided reviews, the primary reasons users express dissatisfaction with other music streaming platforms are:
1. Limited features and functionality compared to Spotify.
2. Poor user interface and overall design of the competitor's app.
3. Limited variety of music and lack of features for discovering new music.
4. Issues with the competitor's app causing it to become unusable on certain devices

SystemExit: 

c:\Users\irfan\Anaconda3\envs\llama\lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [37]:
chat_history

[('What are the specific features or aspects that users appreciate the most in our application?',
  "Based on the two reviews provided, users appreciate the following features or aspects of your application:\n1. Minimal ads: Both reviewers mention that the app has minimal ads, which is a significant aspect they appreciate.\n2. Great lists: Reviewers appreciate the quality of the lists provided by the app, which they find helpful.\n3. Cost: The second reviewer mentions that they would like to see a lower cost of subscription, which suggests that they appreciate the app's pricing model."),
 ('In comparison to our application, which music streaming platform are users most likely to compare ours with?',
  'Based on the reviews, users most often compare our application with Pandora.'),
 ('What are the primary reasons users express dissatisfaction with Spotify?',
  "Based on the provided reviews, the primary reasons users express dissatisfaction with other music streaming platforms are:\n1. 